In [1]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-gemini
!pip install llama-index-llms-gemini
!pip install -q llama-index google-generativeai

In [2]:
%env GOOGLE_API_KEY=...


env: GOOGLE_API_KEY=...


In [3]:
from google.colab import userdata
import os

# Retrieve your API key
api_key = userdata.get('Gemini_API')
os.environ["GOOGLE_API_KEY"] = api_key

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
from llama_index.core import SimpleDirectoryReader
#documents = SimpleDirectoryReader(input_files=['llama2.pdf']).load_data()
documents = SimpleDirectoryReader("/content/sample_data/Dermatology")
documents = documents.load_data()


In [6]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.core import Settings

In [7]:
# Setting global parameter
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5") # set the embedding model
Settings.llm = Gemini(model_name="models/gemini-pro")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=10)

# global
from llama_index.core import Settings

Settings.text_splitter = text_splitter

In [17]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, transformations=[text_splitter])

In [18]:
index.storage_context.persist(persist_dir="/blogs")

In [19]:
from llama_index.core import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="/blogs")

# load index
index = load_index_from_storage(storage_context)

In [20]:
template = """
You are a knowledgeable and precise assistant specialized in question-answering tasks, particularly from Dermatology resources.
Your goal is to provide accurate, concise, and contextually relevant answers based on the given information.

Instructions:

Comprehension and Accuracy: Carefully read and comprehend the provided context from the article to ensure accuracy in your response.
Conciseness: Deliver the answer in no more than three sentences, ensuring it is concise and directly addresses the question.
Truthfulness: If the context does not provide enough information to answer the question, clearly state, "I don't know."
Contextual Relevance: Ensure your answer is well-supported by the retrieved context and does not include any information beyond what is provided.

Remember if no context is provided please say you don't know the answer
Here is the question and context for you to work with:

\nQuestion: {question} \nContext: {context} \nAnswer:"""


from llama_index.core.prompts import PromptTemplate

prompt_tmpl = PromptTemplate(
    template=template,
    template_var_mappings={"query_str": "question", "context_str": "context"},
)


In [21]:
from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine



# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template":prompt_tmpl}
)

In [22]:
## Input
response = query_engine.query("What are some tips for caring for my sensitive skin, especially on my face?")
print(response)

- Use soap-free cleansers and moisturizers to avoid irritation.
- Avoid products containing antibacterial or deodorant ingredients, alcohol, retinoids, or alpha-hydroxy acids.
- Use face powder, silicone-based foundation, black eyeliner and mascara, and pencil eyeliner and eyebrow fillers.


In [14]:
## Input
response = query_engine.query("How is a skin disease diagnosed?")
print(response)

## Output
# According to Table A.2 in the provided context, LLaMA comes in a range of parameter sizes—7B, 13B, and 70B—as well as pretrained and fine-tuned variations.

A healthcare provider can often diagnose a skin disease by visually examining your skin. If a visual examination is not conclusive, tests such as a biopsy, culture, skin patch test, black light examination, diascopy, dermoscopy, or Tzanck test may be used.


In [15]:
## Input
response = query_engine.query("How are skin disease treated?")
print(response)

Skin diseases can be treated with various methods depending on the condition. These treatments may include antibiotics, antihistamines, laser skin resurfacing, medicated creams, ointments or gels, moisturizers, oral medications, steroid pills, creams or injections, and surgical procedures.


In [23]:
## Input
response = query_engine.query("What is Hemangiomas?")
print(response)

Hemangiomas are harmless tumors that usually appear on the neck, head, or face of infants. They begin as small scratches or bumps that eventually bulge out and become larger growths.
